In [1]:
import os
import json
import pandas as pd
import traceback

In [6]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv

In [7]:
load_dotenv()

True

In [8]:
KEY=os.getenv("OPENAI_API_KEY")

In [38]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo",temperature=0.5)

In [39]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [40]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [41]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [42]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [43]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

In [44]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [45]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)


In [46]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)


In [47]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [48]:
file_path=r"C:\Users\sodatta\OneDrive - ALLEGIS GROUP\genai_mcq-generator\data.txt"

In [49]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [50]:
print(TEXT)

The Mach–Zehnder interferometer shows that photons can exhibit wave-like interference.
For many years, the fields of quantum mechanics and computer science formed distinct academic communities.[3] Modern quantum theory developed in the 1920s to explain the wave–particle duality observed at atomic scales,[4] and digital computers emerged in the following decades to replace human computers for tedious calculations.[5] Both disciplines had practical applications during World War II; computers played a major role in wartime cryptography,[6] and quantum physics was essential for the nuclear physics used in the Manhattan Project.[7]

As physicists applied quantum mechanical models to computational problems and swapped digital bits for qubits, the fields of quantum mechanics and computer science began to converge. In 1980, Paul Benioff introduced the quantum Turing machine, which uses quantum theory to describe a simplified computer.[8] When digital computers became faster, physicists faced a

In [51]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [52]:
NUMBER=5 
SUBJECT="quantum computing"
TONE="simple"

In [53]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The Mach–Zehnder interferometer shows that photons can exhibit wave-like interference.
For many years, the fields of quantum mechanics and computer science formed distinct academic communities.[3] Modern quantum theory developed in the 1920s to explain the wave–particle duality observed at atomic scales,[4] and digital computers emerged in the following decades to replace human computers for tedious calculations.[5] Both disciplines had practical applications during World War II; computers played a major role in wartime cryptography,[6] and quantum physics was essential for the nuclear physics used in the Manhattan Project.[7]

As physicists applied quantum mechanical models to computational problems and swapped digital bits for qubits, the fields of quantum mechanics and computer science began to converge. In 1980, Paul Benioff introduced the quantum Turing machine, which uses q

In [54]:

print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:3010
Prompt Tokens:2318
Completion Tokens:692
Total Cost:0.004861


In [55]:
response

{'text': 'The Mach–Zehnder interferometer shows that photons can exhibit wave-like interference.\nFor many years, the fields of quantum mechanics and computer science formed distinct academic communities.[3] Modern quantum theory developed in the 1920s to explain the wave–particle duality observed at atomic scales,[4] and digital computers emerged in the following decades to replace human computers for tedious calculations.[5] Both disciplines had practical applications during World War II; computers played a major role in wartime cryptography,[6] and quantum physics was essential for the nuclear physics used in the Manhattan Project.[7]\n\nAs physicists applied quantum mechanical models to computational problems and swapped digital bits for qubits, the fields of quantum mechanics and computer science began to converge. In 1980, Paul Benioff introduced the quantum Turing machine, which uses quantum theory to describe a simplified computer.[8] When digital computers became faster, physi

In [57]:
quiz=response.get("quiz")

In [61]:
quiz=json.loads(quiz)

In [62]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [63]:
quiz_table_data

[{'MCQ': 'What concept did the Mach–Zehnder interferometer demonstrate about photons?',
  'Choices': 'a: Particle-like behavior | b: Wave-like interference | c: Quantum entanglement | d: Quantum superposition',
  'Correct': 'b'},
 {'MCQ': 'Who introduced the quantum Turing machine in 1980?',
  'Choices': 'a: Paul Benioff | b: Richard Feynman | c: Yuri Manin | d: Charles Bennett',
  'Correct': 'a'},
 {'MCQ': 'Which algorithm in 1994 demonstrated a quantum speedup for unstructured search problems?',
  'Choices': "a: Deutsch's algorithm | b: Bernstein–Vazirani algorithm | c: Simon's algorithm | d: Grover's algorithm",
  'Correct': 'd'},
 {'MCQ': 'When did Google AI and NASA announce achieving quantum supremacy with a 54-qubit machine?',
  'Choices': 'a: 2015 | b: 2017 | c: 2019 | d: 2021',
  'Correct': 'c'},
 {'MCQ': 'What was the major obstacle to practical quantum computers that researchers at Harvard potentially removed in December 2023?',
  'Choices': 'a: Quantum entanglement | b: Qua

In [67]:
quiz=pd.DataFrame(quiz_table_data)

In [69]:
quiz.to_csv("quantam_computing.csv")